In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report,confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
train = pd.read_csv('data/Train.csv')

In [ ]:
# log-transform of twi columns
for col in train.columns:
    if 'twi' in col:
        train[col+'_log'] = np.log(train[col])

In [ ]:
# select all columns of specific features
selected_cols = []
selected_features = ['slope', 'procurv', 'twi_log', 'geology', 'sdoif']
for col in train.columns:
    for feature in selected_features:
        if feature in col:
            selected_cols.append(col)

In [ ]:
# select columns used for x and y
X = train[selected_cols]
y = train.Label

# Split data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.3, random_state=2022, stratify=y)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [ ]:
# Train model
model = LogisticRegression(random_state = 2022, max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_val)

# Check the auc score of the model
print(f'Logistic Regression F1 score on the X_test is: {f1_score(y_val, y_pred)}\n')

# print classification report
print(classification_report(y_val, y_pred))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
fig, ax = plt.subplots(figsize=(15,7))
disp.plot(ax=ax)
plt.show()

In [ ]:
y_pred_proba = model.predict_proba(X_val)
y_val_array = y_val.to_numpy()
fpr, tpr, _ = roc_curve(y_val_array,  y_pred_proba[:,1])
auc = roc_auc_score(y_val_array, y_pred_proba[:,1])
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()